In [2]:
import json
import gzip
import numpy as np
import pandas as pd
from typing import Dict, Text

import tensorflow as tf
import tensorflow_recommenders as tfrs

In [3]:
data = pd.read_csv('../Notebook files/meta_Movies_and_TV_mini.csv')

In [4]:
data = data.drop('Unnamed: 0', axis = 1)
data.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"['Movies & TV', 'Movies']",NaN,[],NaN,Understanding Seizures and Epilepsy,[],NaN,NaN,[],"886,503 in Movies & TV (",[],Movies & TV,NaN,NaN,NaN,0000695009,[],[],NaN
1,"['Movies & TV', 'Movies']",NaN,[],NaN,Spirit Led&mdash;Moving By Grace In The Holy S...,[],NaN,NaN,[],"342,688 in Movies & TV (",[],Movies & TV,NaN,NaN,NaN,0000791156,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,NaN
2,"['Movies & TV', 'Movies']",NaN,['Disc 1: Flour Power (Scones; Shortcakes; Sou...,NaN,My Fair Pastry (Good Eats Vol. 9),[],NaN,Alton Brown,[],"370,026 in Movies & TV (",[],Movies & TV,NaN,NaN,NaN,0000143529,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,NaN
3,"['Movies & TV', 'Movies']",NaN,['Barefoot Contessa Volume 2: On these three d...,NaN,"Barefoot Contessa (with Ina Garten), Entertain...","['B002I5GNW4', 'B005WXPVMM', 'B009UY3W8O', 'B0...",NaN,Ina Garten,[],"342,914 in Movies & TV (","['B002I5GNW4', '0804187045', 'B009UY3W8O', '06...",Movies & TV,NaN,NaN,$74.95,0000143588,[],[],NaN
4,"['Movies & TV', 'Movies']",NaN,['Rise and Swine (Good Eats Vol. 7) includes b...,NaN,Rise and Swine (Good Eats Vol. 7),"['B000P1CKES', 'B000NR4CRM']",NaN,Alton Brown,[],"351,684 in Movies & TV (",['B0015SVNXY'],Movies & TV,NaN,NaN,NaN,0000143502,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,NaN


In [5]:
data.columns.values

array(['category', 'tech1', 'description', 'fit', 'title', 'also_buy',
       'tech2', 'brand', 'feature', 'rank', 'also_view', 'main_cat',
       'similar_item', 'date', 'price', 'asin', 'imageURL',
       'imageURLHighRes', 'details'], dtype=object)

In [6]:
rating = np.random.randint(low = 1, high = 6, size = len(data)).astype('object')

rating = pd.Series(rating, name ='rating')

data = data.join(rating)

In [7]:
reviewerID = np.arange(len(data)).astype('object')
reviewerID = pd.Series(reviewerID, name = 'reviewerID')
data = data.join(reviewerID)

In [8]:
cols_used = ['asin', 'title', 'reviewerID', 'rating']
data = data[cols_used]
data.head()

,asin,title,reviewerID,rating
0,0000695009,Understanding Seizures and Epilepsy,0,3
1,0000791156,Spirit Led&mdash;Moving By Grace In The Holy S...,1,1
2,0000143529,My Fair Pastry (Good Eats Vol. 9),2,5
3,0000143588,"Barefoot Contessa (with Ina Garten), Entertain...",3,2
4,0000143502,Rise and Swine (Good Eats Vol. 7),4,3


In [9]:
combined = data.drop_duplicates(['asin', 'reviewerID'])
user_movies_matrix = combined.pivot(index = 'reviewerID', columns = 'asin', values = 'rating')
#Fill missing values with 0.
user_movies_matrix.fillna(0, inplace = True)

user_movies_matrix.head()

asin,0000143502,0000143529,0000143561,0000143588,0000695009,000073991X,0000791156,000107461X,0001421409,0001485423,...,6301720385,6301720407,6301720555,6301720571,630172058X,6301722000,6301723759,6301723775,630172383X,6301723864
reviewerID,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
data.title.nunique()

4994

In [14]:
# user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token = None)
# user_ids_vocabulary.adapt(data['reviewerID'])

# # movie_id_vocabulary = tf.keras.layers.StringLookup(output_mo)
# # movie_id_vocabulary.adapt(data['title'])

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [58]:
data.reviewerID.dtype

dtype('O')

In [60]:
data.describe()

,asin,title,reviewerID,rating
count,5000,5000,5000,5000
unique,5000,4994,5000,5
top,0783210973,Fire VHS,0,1
freq,1,2,1,1023


In [62]:
data.dtypes

asin          object
title         object
reviewerID    object
rating        object
dtype: object